In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np
from sklearn.metrics import roc_curve, accuracy_score
from scipy.interpolate import interp1d
from scipy.optimize import brentq
import matplotlib.pyplot as plt
import os   

#from tqdm import tqdm

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
import tensorflow as tf

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")


In [ ]:
# Define the means and standard deviations for normalization
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Function to normalize data
def preprocess_input(x):
    x /= 255.0
    x -= mean
    x /= std
    return x
def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides, name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides, name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(name=bn_name_base + '1')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(name=bn_name_base + '2c')(x)

    x = Add()([x, input_tensor])
    x = Activation('relu')(x)
    return x

def ResNet50(input_shape=(224, 224, 3), classes=2, dropout_rate=0.5):  # Default dropout rate set to 0.5
    img_input = Input(shape=input_shape)
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(classes, activation='softmax', name='fc')(x)  # Two units for softmax
    model = Model(inputs=img_input, outputs=x, name='resnet50')
    return model
model = ResNet50(input_shape=(224, 224, 3), classes=2, dropout_rate=0.7)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint('model_best_checkpoint.h5', save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001, mode='min')

# Data augmentation
train_transform = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

val_test_transform = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

image_size = (224, 224)
# Create ImageDataGenerators for training and validation/test
train_transform = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=preprocess_input  # Apply normalization using preprocess_input
)

val_test_transform = ImageDataGenerator(
    preprocessing_function=preprocess_input  # Apply normalization using preprocess_input
)

In [ ]:
train_path = r'C:\Mine\Work\Uzi\Signature_Verification\train'
val_path = r'C:\Mine\Work\Uzi\Signature_Verification\validation'
test_path = r'C:\Mine\Work\Uzi\Signature_Verification\test'

In [ ]:
# Create data generators
train_data = train_transform.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical'  # Changed to categorical
)

val_data = val_test_transform.flow_from_directory(
    val_path,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical'  # Changed to categorical
)

test_data = val_test_transform.flow_from_directory(
    test_path,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical'  # Changed to categorical
)

In [ ]:
# Training Loop
epochs = 8
model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    steps_per_epoch=len(train_data),
    validation_steps=len(val_data),
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

In [ ]:
# Calculating the number of batches in the test set
test_sample_count = test_data.samples
batch_size = test_data.batch_size
num_batches = np.ceil(test_sample_count / batch_size)

# Testing loop with tqdm
all_labels = []
all_scores = []

for i in tqdm(range(int(num_batches))):
    inputs, labels = next(test_data)
    probabilities = model.predict(inputs)
    all_scores.extend(probabilities.ravel())
    all_labels.extend(labels)

# Convert lists to numpy arrays for further processing
all_labels = np.array(all_labels)
all_scores = np.array(all_scores)

In [ ]:

# Accuracy, EER, and TAR at specific FAR calculation
def calculate_metrics(labels, scores, far_target=1e-3):
    fpr, tpr, thresholds = roc_curve(labels, scores)
    eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    far_index = np.where(fpr <= far_target)[0][-1]
    tar_at_far = tpr[far_index]
    accuracy = accuracy_score(labels, scores > 0.5)
    return accuracy, eer, tar_at_far

accuracy, eer, tar_at_far = calculate_metrics(all_labels, all_scores)
print(f'Accuracy: {accuracy:.4f}, EER: {eer:.4f}, TAR at FAR={1e-3}: {tar_at_far:.4f}')

In [ ]:
model.save('ResNet50')


In [ ]:
import matplotlib.pyplot as plt

def visualize_misclassified_samples_with_filenames(test_data, model, num_images=10):
    misclassified_images = []
    misclassified_predictions = []
    true_labels = []
    filenames = []

    for i in range(len(test_data)):
        inputs, labels = next(test_data)
        predictions = model.predict(inputs)
        predicted_classes = np.argmax(predictions, axis=1)
        true_classes = np.argmax(labels, axis=1)

        for j, (img, pred, true) in enumerate(zip(inputs, predicted_classes, true_classes)):
            if pred != true:
                misclassified_images.append(img)
                misclassified_predictions.append(pred)
                true_labels.append(true)
                filenames.append(test_data.filenames[i * test_data.batch_size + j])

                if len(misclassified_images) >= num_images:
                    break
        if len(misclassified_images) >= num_images:
            break

    # Plotting
    plt.figure(figsize=(15, 5))
    for k, (img, pred, true, fname) in enumerate(zip(misclassified_images, misclassified_predictions, true_labels, filenames)):
        plt.subplot(2, num_images // 2, k+1)
        plt.imshow(img)
        plt.title(f'File: {fname}\nPred: {pred}, True: {true}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Example usage
visualize_misclassified_samples_with_filenames(test_data, model)
